# <u>Four Mile Map Features and Calculations</u>
## by Habib Bravo-Ruiz for Final Project of Geography 489 at Penn State University

### 1. Overview
This notebook can be used to generate and illustrate data for sensitive environments within four miles of a Puerto Rico site. The generated data can be used for Hazardous Ranking System (HRS) evaluations. The HRS is a system that the Environmental Protection Agency (EPA) uses to score and evaluate potential threats to public health and the environment posed by uncontrolled releases or threatened releases of hazardous substances, pollutants, or contaminants. It is the primary screening tool for determining whether a site is to be included on the Nation Priority List (NPL), EPA’s list of sites that are priorities for further investigation and, if necessary, response action under the Comprehensive Environmental Response, Compensation, and Liability Act (CERCLA).

An HRS score for a site is determined by evaluating four pathways: Ground water migration; Surface water migration; Soil exposure and Subsurface Intrusion; and Air migration. The scoring system for each pathway is based on several individual factors grouped into three-factor categories: (1) likelihood of release (or, for the Soil Exposure and Subsurface Intrusion pathway, the likelihood of exposure); (2) waste characteristics; and (3) targets. The four mile maps specifically deal with targets. Targets consist of human populations, sensitive environments, fisheries, and resources that can potentially be affected by a releases from a contaminated site.

The code has been implemented and tested with Python 3.9.11.

Python packages used:

* From the Python standard library:
    * os (https://docs.python.org/3/library/os.html)
    * sys (https://docs.python.org/3/library/sys.html)
    * re (https://docs.python.org/3/library/re.html)
    * csv (https://docs.python.org/3/library/csv.html)
    * zipfile (https://docs.python.org/3/library/zipfile.html)
* arcgis - ArcGIS API for Python (https://developers.arcgis.com/python/)
* arcpy (https://www.esri.com/en-us/arcgis/products/arcgis-python-libraries/libraries/arcpy)
* pandas (https://pypi.python.org/pypi/pandas)- 
* urllib3 (https://urllib3.readthedocs.io/en/stable/)

### 2. Preparation
To run this notebook, the above Python packages must be installed and the input data must be saved in a new folder (referred to as the workspace folder in the following). Furthermore, four input variables in Section 2.2.1 of this notebook need to be adapted.

#### 2.1 Data for this project
The following files are required to run this notebook:
* FourMileData.gdb - This file contains the feature classes of the sensitive environments for the whole island of Puerto Rico.

#### 2.2 Importing packages and preparing notebook
The following subsections contain some first Python code to initialize the project, starting with some important input variables that need to adapted for your purposes.

##### 2.2.1 Setup Input Variables to be Adapted by the User
The workspace variable should point to the folder in which the input data is stored. **Please adapt the path accordingly.**

In [1]:
# Path to geodatabase containing the input features
workspace = r"C:\PSU\Geog489\Final Project\Deliverables\Aprx\FourMileData.gdb"

# Path to the geodatabase where the output features will be stored
# If the geodatabase does not exist, the script will create one for you
outputGDB = r"C:\PSU\Geog489\Final Project\Deliverables\Aprx\FinalTest.gdb"

# Folder where you want the output CSV to be stored
outputCSV = r"C:\PSU\Geog489\Final Project\Deliverables\Aprx"

# Your ArcGIS Online username
usernameAGOL = "hhb5067_pennstate"

##### 2.2.2 Setup Input Variables to be Adapted by the User
The below defines the intial variables used by the script. The names within the lists are the names of the sensitive environment feature classes in the workspace geodatabase.

In [2]:
mainTargetEnvsFCs = ["EN_and_NT_Species", "Natural_Areas", "Critical_Wildlife", "Coastal_Barriers"]
otherTargetEnvsFCs = ["Wetlands", "Census_2010"]

##### 2.2.3 Import General Python Packages Needed and Set Arcpy Settings
Python packages that we will be used in different places throught the notebook are imported and some arcpy settings are defined.

In [3]:
import arcgis, arcpy, os, pandas
import fourmilefunctions_jupyter as functs

arcpy.env.overwriteOutput = True
arcpy.env.workspace = workspace

##### 2.2.4 Check Geodatabase Status
Below a check is peformed to see if the geodatabase defined by the user in Section 2.2.1 exists. If the geodatabase exists, it provides the user options regarding what to with the geodatabase. If it does not exists, it creates the geodatabase.

In [4]:
gdbTest = functs.checkOutputGDB(outputGDB)

Geodatabase already exists. Do you want to overwrite it? y


##### 2.2.5 Create a Point Feature Class for the Input Site
Below a point feature class is created based on user input. The script will show a text box where the user has to type the site name. After typing the site name press enter and another text box will show for the longitude. After typing the longitude press enter and the last box will appear for the latitude. Both longitude and latitude have to be entered in decimal degrees. The coordinates must be within the boundaries of mainland Puerto Rico. Can you the following for example. Lat: 18.456, Long: -66.855

In [5]:
sitesFC = functs.createInputData(gdbTest)
siteList = functs.getSites(sitesFC)

Please enter the name of the site: TestSite
Please enter site longitude in decimal degrees: -66.85
Please enter site latitude in decimal degrees: 18.45


### 3. Processing Sensitive Environment Data
In the next sections, the sensitive environment feature classes in the workspace are processed based on the location of the input site. 

#### 3.1 Geoprocessing of Sensitive Environments
The next function creates 0.25-, 0.5-, 1-, 2-, 3-, 4-mi buffer rings around the input site.

In [6]:
siteRingsFCsList = functs.createBufferRings(sitesFC, siteList)

The first function below clips all the sensitve environment feature classes except the wetlands and population. This function also returns a list the of clipped feature classes that were generated. The idenBaseMapFiles functions performs clip and identity on wetlands and population feature classes. Identitiy is required for calculations. This function also returns a list of identity feature classes.

In [7]:
mainTargetEnvsFCsClip = [functs.clipBasemapFiles(fcs, siteList, siteRingsFCsList) for fcs in mainTargetEnvsFCs]
otherTargetEnvsFCsIden = [functs.idenBasemapFiles(fcs, siteList, siteRingsFCsList) for fcs in otherTargetEnvsFCs]

#### 3.2 Population and Wetland Calculations
The below function performs wetland area and population calculations per buffer ring using their identity feature classes created above and returns a csv file.Then a pandas dataframe is created using the csv file to present the data to the user.

In [8]:
calcData = functs.targetCalculations(siteList, otherTargetEnvsFCsIden, outputCSV)

In [9]:
targetCalculationsDF = pandas.read_csv(calcData)
targetCalculationsDF

,Site Name,Distance Ring (mi),Wetland Area (acres),Population
0,TestSite,0.25,0.0,130.0
1,TestSite,0.50,0.0,587.0
2,TestSite,1.00,5.5,2431.0
3,TestSite,2.00,208.1,9995.0
4,TestSite,3.00,413.3,19211.0
5,TestSite,4.00,287.6,16707.0


### 4. Visualization of the Processing Results with ArcGIS Online
This details how an interactive map representation of the final data is added to the notebook with ESRI ArcGIS API for Python.

#### 4.1 Create Zipped Output Geodatabase
The copyFCs function copies all the resulting feature classes that need to be displayed. Then a zipfile version of the output geodatabase is created to be uploaded to ArcGIS Online.

In [10]:
functs.copyFCs(siteList, gdbTest)
functs.createZipfile(gdbTest)

#### 4.2 Upload and Publish to ArcGIS Online
In this section the ArcGIS API for Python is used to establish a connection to a PSU AGOL account and publish the zipped geodatabase as a feature service that will be use in a map visualization. First, a GIS object connected to the AGOL account is created.

In [11]:
import urllib3 # urllib3 is imported to prevent getting InsecureRequestWarning after signing in
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from arcgis.gis import GIS
gis = GIS("https://pennstate.maps.arcgis.com", client_id="lDSJ3yfux2gkFBYc")

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://pennstate.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=lDSJ3yfux2gkFBYc&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=6myU3N47M2Qv5aQJ5BgyLeOlbl9Wg9&allow_verification=false
Enter code obtained on signing in using SAML: ········


The following code checks that there is no output geodatabase feature service with the already on AGOL from previous runs of the notebook.

In [13]:
# Get AGOL username
user = gis.users.get(usernameAGOL)

# Search for content in your gis with a query built from title, owner and item type
def searchAGOL(title, owner, itemType):
    return gis.content.search(query="title:"+title+" owner:"+owner, item_type=itemType)

# Test whether items exist on AGOL for given title, owner, and item type and if so, delete them from AGOL
def deleteIfExistsOnAGOL(title, owner, itemType):
    # Search item
    result = searchAGOL(title, owner, itemType)
    print("Found items for title="+title+", owner="+owner+", itemType="+itemType+":")
    print(result)
    # Delete items found
    for item in result:
        item.delete()
        print("Item " + item.title + " has been deleted.")

# Delete existing output shapefiles and feature layers on AGOL
deleteIfExistsOnAGOL(os.path.basename(os.path.splitext(outputGDB)[0]), user.username, "File geodatabase")
deleteIfExistsOnAGOL(os.path.basename(os.path.splitext(outputGDB)[0]), user.username, "Feature Service")

Found items for title=FinalTest, owner=hhb5067_pennstate, itemType=File geodatabase:
[]
Found items for title=FinalTest, owner=hhb5067_pennstate, itemType=Feature Service:
[]


Next the zipped geodatabase is uploaded to AGOL using the gis.content.add() method.

In [14]:
finalOutputGDB = gis.content.add({"type":"File Geodatabase"}, os.path.splitext(gdbTest)[0] + ".zip")

Then the uploaded geodatabase is published as a feature service by calling the publish() method.

In [15]:
finalOutputGDBFeatServ = finalOutputGDB.publish()

Next the AGOL content is checked to confirm that the geodatabase and its related feature service were uploaded.

In [16]:
print(searchAGOL(os.path.basename(os.path.splitext(gdbTest)[0]), usernameAGOL, "File Geodatabase"))
print(searchAGOL(os.path.basename(os.path.splitext(gdbTest)[0]), usernameAGOL, "Feature Service")) 

[<Item title:"FinalTest" type:File Geodatabase owner:hhb5067_pennstate>]
[<Item title:"FinalTest" type:Feature Layer Collection owner:hhb5067_pennstate>]


#### 4.3 Final Interactive Map Visualization
In this section a map widget is created to show the sensitive environments within four miles the input site.

In [17]:
# Create feature service object
uploadedItem = searchAGOL(os.path.basename(os.path.splitext(gdbTest)[0]), usernameAGOL, "Feature Service")
uploadedItemFL = uploadedItem[0]

# Create layer objects for each feature in the feature service
pointLyr = uploadedItemFL.layers[0]
ringLyr = uploadedItemFL.layers[1]
endangeredSpeciesLyr = uploadedItemFL.layers[2]
naturalAreasLyr = uploadedItemFL.layers[3]
criticalWildlifeLyr = uploadedItemFL.layers[4]
coastalLyr = uploadedItemFL.layers[5]
wetlandsLyr = uploadedItemFL.layers[6]

# Get for each feature in the feature service from the layer objects
pointUrl = pointLyr.url
ringUrl = ringLyr.url
endangeredSpeciesUrl = endangeredSpeciesLyr.url
naturalAreasUrl = naturalAreasLyr.url
criticalWildlifeUrl = criticalWildlifeLyr.url
coastalUrl = coastalLyr.url
wetlandsUrl = wetlandsLyr.url

# Display feature service info
uploadedItemFL

<Item title:"FinalTest" type:Feature Layer Collection owner:hhb5067_pennstate>

The below code creates a map widged focused on Puerto Rico.

In [18]:
# Create map widget object with zoom
finalMap = gis.map("Puerto Rico", 9)

# Center the extent in the map widget object to NE USA
finalMap.center = [18.2,-66.5]

# Add the map widget to notebook
finalMap

MapView(layout=Layout(height='400px', width='100%'))

Next the basemap is made gray to make the features that will be added easier to see.   

In [19]:
# Change basemap style
finalMap.basemap = "gray-vector"

Last, the features are added to the map widget using the add_layer() function and their url.

In [20]:
#Add site layers to final map
finalMap.add_layer({"type":"FeatureLayer",
                    "url": endangeredSpeciesUrl,
                    "opacity": 0.5})

finalMap.add_layer({"type":"FeatureLayer", 
                    "url": naturalAreasUrl,
                    "opacity": 0.5})

finalMap.add_layer({"type":"FeatureLayer", 
                    "url": criticalWildlifeUrl,
                    "opacity": 0.5})

finalMap.add_layer({"type":"FeatureLayer", 
                    "url": wetlandsUrl})

finalMap.add_layer({"type":"FeatureLayer", 
                    "url": coastalUrl,
                    "opacity": 0.5})

finalMap.add_layer({"type":"FeatureLayer", 
                    "url": ringUrl,
                    "opacity": 0.5})

finalMap.add_layer({"type":"FeatureLayer", 
                    "url": pointUrl,
                    "opacity": 0.5})